In [4]:
ENV["IJULIA_DEBUG"]=true
import Pkg; Pkg.add("JuMP")
Pkg.add("Gurobi")
using Gurobi

using DataFrames, CSV
using JuMP

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


In [5]:
centers = CSV.read("centers.csv", DataFrame, header=0)
landfills = CSV.read("landfills.csv", DataFrame, header=0)
q = CSV.read("q.csv", DataFrame, header=0)
stations = CSV.read("stations.csv", DataFrame, header=0)

centers = Matrix(centers)
landfills = Matrix(landfills)
q = Matrix(q)
stations = Matrix(stations)

50×2 Matrix{Float64}:
 85.45    19.15
  5.8242  68.119
 99.657   72.79
 63.366   92.143
 41.421   73.084
 61.369   50.405
 99.738   14.051
 63.092   74.788
 49.123    8.6421
  6.4015   1.2557
 83.286   14.437
 38.906   35.617
 72.829   30.488
  ⋮       
  1.5279  51.627
  1.4406  32.745
 51.475   39.177
 28.904   36.184
 27.421   18.035
  6.9819  42.147
  7.478   84.236
 86.416    6.6111
 78.93    99.13
 35.57    52.772
  4.8863  50.596
 34.388   22.904

In [27]:
mp = Model(Gurobi.Optimizer)

# Add variables
@variable(mp, x_ij[1:50, 1:15])  # Waste transported from center i to landfill j directly
@variable(mp, x_ik[1:50, 1:50])  # Waste transported from center i to station k first
@variable(mp, x_kj[1:50, 1:15])  # Waste transported from station k to landfill j
@variable(mp, z[1:15], Int)  # if landfill j is built
@variable(mp, s[1:50], Int)  # if compact station k is built

Set parameter Username
Academic license - for non-commercial use only - expires 2023-09-22


50-element Vector{VariableRef}:
 s[1]
 s[2]
 s[3]
 s[4]
 s[5]
 s[6]
 s[7]
 s[8]
 s[9]
 s[10]
 s[11]
 s[12]
 s[13]
 ⋮
 s[39]
 s[40]
 s[41]
 s[42]
 s[43]
 s[44]
 s[45]
 s[46]
 s[47]
 s[48]
 s[49]
 s[50]

In [38]:
# Define distance parameters
d_ij = zeros(50, 15)
for i in 1:50
    for j in 1:15
        d_ij[i, j] = sqrt((centers[i,1] - landfills[j,1])^2 + (centers[i,2] - landfills[j,2])^2)
    end
end
d_ik = zeros(50, 50)
for i in 1:50
    for k in 1:50
        d_ik[i, k] = sqrt((centers[i,1] - stations[k,1])^2 + (centers[i,2] - stations[k,2])^2)
    end
end
d_kj = zeros(50, 15)
for k in 1:50
    for j in 1:15
        d_kj[k, j] = sqrt((stations[k,1] - landfills[j,1])^2 + (stations[k,2] - landfills[j,2])^2)
    end
end

# objective function
@objective(mp, Min, sum(d_ij[i, j] * x_ij[i, j] for i in 1:50, j in 1:15) + sum(d_ik[i, k] * x_ik[i,k] for i in 1:50, k in 1:50) + sum(d_kj[k, j] * x_kj[k, j] * 0.5 for k in 1:50, j in 1:15) + sum(10000 * s[k] for k in 1:50));

In [39]:
# Ensure all of the waste leaves the centers
for i in 1:50
    @constraint(mp, sum(x_ij[i,j] for j in 1:15) + sum(x_ik[i,k] for k in 1:50) .>= q[i]);
end

# Ensure that waste can only go to a landfill if it is built
for j in 1:15
    for i in 1:50
        @constraint(mp, x_ij[i,j] .<= 1000000 .* z[j]);
    end
    for k in 1:50
        @constraint(mp, x_kj[k,j] .<= 1000000 .* z[j]);
    end
end

# Ensure that waste can only go to a station if it is built
# Ensure that if waste goes in a station it must come out
# Ensure that no more than 2000 tons is processed in each station
for k in 1:50
    for i in 1:50
        @constraint(mp, x_ik[i,k] .<= 1000000 .* s[k]);
    end
    
    @constraint(mp, sum(x_ik[i,k] for i in 1:50) - sum(x_kj[k, j] for j in 1:15) .<= 0.);
    
    @constraint(mp, sum(x_ik[i,k] for i in 1:50) .<= 2000);
end

# Can only build 5 landfills
@constraint(mp, sum(z[j] for j in 1:15) .<= 5)

@constraint(mp, x_ij .>= zeros(50, 15));
@constraint(mp, x_ik .>= zeros(50, 50));
@constraint(mp, x_kj .>= zeros(50, 15));

In [40]:
optimize!(mp)

objective_value(mp)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 16302 rows, 4065 columns and 42030 nonzeros
Model fingerprint: 0xd2a0bd79
Variable types: 4000 continuous, 65 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  Objective range  [1e+00, 1e+04]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+00, 2e+03]

MIP start from previous solve produced solution with objective 808643 (0.01s)
Loaded MIP start from previous solve with objective 808643

Presolve removed 12151 rows and 0 columns
Presolve time: 0.03s
Presolved: 4151 rows, 4065 columns, 17065 nonzeros
Variable types: 4000 continuous, 65 integer (50 binary)

Root relaxation: objective 7.454420e+05, 413 iterations, 0.01 seconds (0.01 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0

808642.7541913454

In [45]:
@show value.(z)
@show value.(s)


# Number of stations built
count = 0
for k in 1:50
    if value.(s)[k] == 1
        count += 1
    end
end

@show count

value.(z) = [0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0]
value.(s) = [0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0]
count = 9


9

In [46]:
# Calculate total waste-miles
miles_tons = sum(d_ij[i, j] * value.(x_ij)[i, j] for i in 1:50, j in 1:15) + sum(d_ik[i, k] * value.(x_ik)[i,k] for i in 1:50, k in 1:50) + sum(d_kj[k, j] * value.(x_kj)[k, j] for k in 1:50, j in 1:15);

@show miles_tons

miles_tons = 876496.7565920389


876496.7565920389

In [43]:
costs = sum(d_ij[i, j] * value.(x_ij)[i, j] for i in 1:50, j in 1:15) + sum(d_ik[i, k] * value.(x_ik)[i,k] for i in 1:50, k in 1:50) + sum(d_kj[k, j] * value.(x_kj)[k, j] * 0.5 for k in 1:50, j in 1:15);

@show costs 

costs = 718642.7541913454


718642.7541913454

In [44]:
total_costs = sum(d_ij[i, j] * value.(x_ij)[i, j] for i in 1:50, j in 1:15) + sum(d_ik[i, k] * value.(x_ik)[i,k] for i in 1:50, k in 1:50) + sum(d_kj[k, j] * value.(x_kj)[k, j] * 0.5 for k in 1:50, j in 1:15) + sum(10000 * value.(s)[k] for k in 1:50);

@show total_costs

total_costs = 808642.7541913454


808642.7541913454